# 🚨🚨🚨🚨 NOT IMPLIMENTED... This is a placeholder for potential brightfield
🚧 WIP 🚧 

# Infer Cell Membrane - part 1

--------------

## OBJECTIVE:  Infer sub-cellular component #1: m  in order to understand interactome 

To measure shape, position, size, and interaction of eight organelles/cellular components Nuclei (NU).  

Dependencies:
CELLMASK and CYTOPLASM inference rely on the Nuclei inference.  Therefore all of the sub-cellular objects rely on the NU segmentation.





# IMPORTS

In [ ]:
# top level imports
from pathlib import Path
import os, sys
from collections import defaultdict

import numpy as np
import scipy

# TODO:  prune the imports.. this is the big set for almost all organelles
# # function for core algorithm
from scipy import ndimage as ndi
import aicssegmentation
from aicssegmentation.core.seg_dot import dot_3d_wrapper, dot_slice_by_slice, dot_2d_slice_by_slice_wrapper, dot_3d
from aicssegmentation.core.pre_processing_utils import ( intensity_normalization, 
                                                         image_smoothing_gaussian_3d,  
                                                         image_smoothing_gaussian_slice_by_slice )
from aicssegmentation.core.utils import topology_preserving_thinning, size_filter
from aicssegmentation.core.MO_threshold import MO
from aicssegmentation.core.utils import hole_filling
from aicssegmentation.core.vessel import filament_2d_wrapper, vesselnessSliceBySlice
from aicssegmentation.core.output_utils import   save_segmentation,  generate_segmentation_contour
                                                 
from skimage import filters
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.morphology import remove_small_objects, binary_closing, ball , dilation, remove_small_holes   # function for post-processing (size filter)
from skimage.measure import label

# # package for io 
from aicsimageio import AICSImage

import napari

### import local python functions in ../infer_subc
sys.path.append(os.path.abspath((os.path.join(os.getcwd(), '..'))))

from infer_subc.core.file_io import (read_input_image, 
                                                                    list_image_files, 
                                                                    export_ome_tiff, 
                                                                    etree_to_dict, 
                                                                    save_parameters, 
                                                                    load_parameters, 
                                                                    export_ndarray)
from infer_subc.core.img import *

%load_ext autoreload
%autoreload 2

test_img_n = 5

# OBJECTIVE :  infer NUCLEI
 

NOTE:  using Allen Cell Segmenter  [Nucleophosmin](https://www.allencell.org/cell-observations/category/nucleophosmin) might be a good generic mechanism.  e.g.
-  [playground_npm1.ipynb](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/lookup_table_demo/playground_npm1.ipynb) and [npm1.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1.py) and [npm1_SR.py](/Users/ahenrie/Projects/Imaging/mcz_subcell/napari/aics-segmentation/aicssegmentation/structure_wrapper/seg_npm1_SR.py) 


## summary of steps


INPUT
- channel 0 from raw data :: raw_nuclei

PRE-PROCESSING
-  scale to min 0, max 1.0
- median Filter window 4

CORE-PROCESSING
  - threshold method minimum cross-entropy.  (Li's method)
    - objects size: 50-400 pixels, 
    - threshold smoothing scale: 1 pixel
    - threshold correction factor: 1.2 (more stringent)
    - lower / upper bounds  (.1,1)
    - log transformed thresholding
    - fill holes


POST-PROCESSING
  - fill holes
  - remove small objects


OUTPUT
- inferred of NUCLEI :: inferer_NU an image of labels export as ome.tiff


> #### Note:  this initial inferred object -- the Nuclei of the brightest cell -- will be used in inferring the Cellmask and Cytoplasm objects.   This is a straightforward procedure, but also note that any inconsistencies will flow into the Cellmask and Cytoplasm objects which in turn affect ALL inferred objects.




In [ ]:
# assumptions:   .czi "unmixed" collections.
#czi_img_folder = f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data"

data_path = Path( f"{os.getenv('HOME')}/Projects/Imaging/mcz_subcell/data")
czi_img_folder = data_path/"raw"

list_img_files = lambda img_folder,f_type: [os.path.join(img_folder,f_name) for f_name in os.listdir(img_folder) if f_name.endswith(f_type)]

img_file_list = list_img_files(czi_img_folder,'.czi')
print(img_file_list[test_img_n])
#Read the data into memeory from the `.czi` files.  (Note: there is also the 2D slice .tif file read for later comparision).  WE will also collect metatdata.


/Users/ahenrie/Projects/Imaging/mcz_subcell/data/raw/ZSTACK_PBTOhNGN2hiPSCs_BR3_N04_Unmixed.czi


In [ ]:

test_img_name = img_file_list[5]

img_data, meta_dict = read_input_image(test_img_name)

raw_meta_data, ome_types = get_raw_meta_data(meta_dict)

# get some top-level info about the RAW data
channel_names = meta_dict['name']
img = meta_dict['metadata']['aicsimage']
scale = meta_dict['scale']
channel_axis = meta_dict['channel_axis']



/opt/anaconda3/envs/napariNEW/lib/python3.9/site-packages/ome_types/_convenience.py:105: FutureWarning: The default XML parser will be changing from 'xmlschema' to 'lxml' in version 0.4.0.  To silence this warning, please provide the `parser` argument, specifying either 'lxml' (to opt into the new behavior), or'xmlschema' (to retain the old behavior).
  d = to_dict(os.fspath(xml), parser=parser, validate=validate)
